In [1]:
import numpy as np
import data
from functions import (filtering, masks)

In [2]:
# load 10 videos into dictionary dset, keys are filenames, values are data object - for Demo
filenum = 10
filepath = 'SampleData'
filenames = ['Block' + str(i+1) + '.tif' for i in range(filenum)]
dset = {}

for filename in filenames:
    dset[filename] = data.Data(filepath, filename)

In [7]:
# calculate M6 on each video [0:9] - for Demo
m_set = {}
for filename in filenames:
    m_set[filename] = dset[filename].moments_images(6)[6]    

In [8]:
m_set[filenames[0]]

Image([[ 146599590365744,   90571191029081,  272101760182672, ...,
          59350132439161,   32492013076433,   11742189854429],
       [ 223073243872876,  207648824920054,   61715281680522, ...,
           9530095449668,   18279365814600,   21174860057859],
       [  77229430498215,  153038091926215,  189051584844149, ...,
          14653929813782,   45109892676569,   18534218455853],
       ...,
       [ 573095670561448,  991148087132016,  938150529138420, ...,
          47006231879300,   30062304763719,   17725361481711],
       [1859973333118466, 1562772890617558, 1248436575875913, ...,
          44847745761565,   37261251153506,   84310458061269],
       [2360888419163300, 2609975722106216, 1673360879587659, ...,
          34032249773485,   47319812368833,   26658044108375]])

In [9]:
nf = masks.gauss1D_mask(shape = (1,21), sigma = 2)
filtering.noise_filter(dset, m_set, nf, filtername = 'noise filter after M6')
dset[filenames[0]].filtered

array([[6.35062902e+13, 4.64063106e+13, 1.06788957e+14, ...,
        1.60361660e+13, 1.15015093e+13, 1.08286780e+13],
       [1.18259012e+14, 1.21103295e+14, 1.51959512e+14, ...,
        6.44740247e+12, 8.90343957e+12, 1.07081540e+13],
       [5.44317778e+13, 6.91845495e+13, 9.66386714e+13, ...,
        7.31817417e+12, 1.51763787e+13, 1.86079791e+13],
       ...,
       [4.61816367e+14, 7.12101976e+14, 7.51933863e+14, ...,
        1.77102182e+13, 1.63993580e+13, 2.18691717e+13],
       [7.08194112e+14, 9.95947123e+14, 9.80837660e+14, ...,
        3.27902111e+13, 2.01624518e+13, 3.07440015e+13],
       [1.17585781e+15, 1.16122366e+15, 1.08206784e+15, ...,
        4.91985089e+13, 2.32157570e+13, 1.78659582e+13]])

In [10]:
dset[filenames[1]].filtered

array([[7.91879711e+13, 6.20580114e+13, 1.30682592e+14, ...,
        1.69021031e+13, 1.28685530e+13, 1.36631781e+13],
       [1.50277851e+14, 1.50978428e+14, 2.18156632e+14, ...,
        8.17838662e+12, 1.08966292e+13, 1.30960081e+13],
       [7.40439600e+13, 8.83395303e+13, 1.28500783e+14, ...,
        9.46669929e+12, 1.61867425e+13, 2.35603296e+13],
       ...,
       [6.53789048e+14, 9.76909820e+14, 9.93978455e+14, ...,
        2.07670370e+13, 2.25981915e+13, 3.47249983e+13],
       [9.39768184e+14, 1.24241431e+15, 1.28203711e+15, ...,
        4.36422087e+13, 2.47892542e+13, 3.63119445e+13],
       [1.47397130e+15, 1.41541316e+15, 1.45069774e+15, ...,
        8.26787959e+13, 2.75154972e+13, 2.26037341e+13]])

In [ ]:
# noise filtering
# TODO: notes on how to determine the filter type / size.
get_series = lambda i,j: [m_set[filename][i,j] for filename in filenames]
noise_filter = gauss1D_mask(shape = (1,21), sigma = 2)
xdim, ydim = np.shape(m_set[filenames[0]])
m_length = len(filenames)
# all the filtered image will be stored in a 3-dimensional (l,x,y) array m_filtered, and can be convreted to a dictionary later.
m_filtered = np.zeros((m_length, xdim, ydim))

for i in range(xdim):
    for j in range(ydim):
        time_series = get_series(i,j)
        m_filtered[:,i,j] = filter1D_same(time_series, noise_filter)
        # filtered_series = filter1D_same(time_series, noise_filter)
        # m_filtered[:,i,j] = filtered_series

# Add filtered images back to data class as a feature
for k in range(m_length):
    dset[filenames[k]].add_filtered(m_filtered[k], 'noise filter after M6')    

In [ ]:
nf = gauss1D_mask(shape = (1,21), sigma = 2)
def noise_filter(data_set, noise_filter, filtername = 'noise filter after M6', filenames = None, return_option = False):
    if filenames is None:
        filenames =[*data_set]
    get_series = lambda i,j: [m_set[filename][i,j] for filename in filenames]
    xdim, ydim = np.shape(m_set[filenames[0]])
    m_length = len(filenames)
    m_filtered = np.zeros((m_length, xdim, ydim))
    for i in range(xdim):
        for j in range(ydim):
            time_series = get_series(i,j)
            m_filtered[:,i,j] = filter1D_same(time_series, noise_filter)
    for k in range(m_length):
        dset[filenames[k]].add_filtered(m_filtered[k], filtername)
        
    if return_option == True:
        return m_filtered

In [ ]:
noise_filter(m_set, nf,return_option=True)

In [ ]:
# TODO: figure out how to import masks under the same function folder
def gauss2D_mask(shape = (3,3), sigma = 0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    mdim, ndim = [(pixel-1) / 2 for pixel in shape]
    y, x = np.ogrid[-mdim:(mdim + 1), -ndim:(ndim + 1)]
    h = np.exp( -(x*x + y*y) / (2*sigma*sigma) )
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

def gauss1D_mask(shape = (1,3), sigma = 0.5):
    return gauss2D_mask(shape, sigma)[0]

In [ ]:
def filter1D_same(time_series, noise_filter):
    '''
    Filter original time_series with noise_filter, and return the filtered_series with the same 
    length as the original sereis.
    Compared to MATLAB results, when the length of the filter and frame number are both even, 
    the filtered result would shift to left by one number. In other cases, results are the same.
    '''
    filtered_series = np.convolve(time_series, noise_filter, 'full')
    filtered_center = len(filtered_series)//2
    original_center = len(time_series)/2
    filtered_series = filtered_series[np.int(filtered_center - np.floor(original_center)):np.int(filtered_center + np.ceil(original_center))]
    return filtered_series